#Neural Decoding - Latent prediction - Progressive idea - Umut
##Written by Gergana Slaveykova s1070004
##Radboud University- B3 Thesis project

# Imports

In [ ]:
#Mount Google Drive to acess data and storing purposes
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installs
! pip install mxnet
! pip install nilearn
! pip install nibabel
!pip install git+https://github.com/nipy/nipy.git


In [ ]:
#Imports
import pickle
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

import nibabel as nib
import numpy as np
from nipy.modalities.fmri.experimental_paradigm import load_paradigm_from_csv_file
from nipy.modalities.fmri.design_matrix import make_dmtx
from nipy.labs.viz import plot_map, cm
import matplotlib.pyplot as plt
import pickle

from __future__ import annotations

import os
from types import ModuleType
from typing import Tuple, Union

from nilearn import masking, plotting
from PIL import Image
from scipy.stats import t, zscore
from sklearn.linear_model import RidgeCV

from sklearn.linear_model import LinearRegression
import h5py
import PIL.Image
from scipy import sparse, stats
from sklearn.kernel_ridge import KernelRidge
from numpy.linalg import svd
import time
from scipy import signal

# Structure


In [ ]:
#Regions of interest
rois = {
        'V1' : 'ROI_V1 = 1',
        'V2' : 'ROI_V2 = 1',
        'V3' : 'ROI_V3 = 1',
        'V4' : 'ROI_V4 = 1',
        'LOC' : 'ROI_LOC = 1',
        'FFA' : 'ROI_FFA = 1',
        'PPA' : 'ROI_PPA = 1'}

rois.keys()

dict_keys(['V1', 'V2', 'V3', 'V4', 'LOC', 'FFA', 'PPA'])

In [ ]:
#Load brain data
x_tr = []
x_te = []
for roi_index, roi in enumerate(rois):
    _roi = np.load(f"/content/drive/MyDrive/GOD-NEW-3p/my_experiment_{roi}_hyperaligned.npy")              # load in roi, all voxels, you want to select a mask
    x_tr.append(_roi[:1200])               # mean over participants, first 1200 are training set, only use mask voxels
    x_te.append(_roi[1200:1250])          # mean over participants, 1200:1250 are test set, only use mask voxels


In [ ]:
#oad latents of images
w_te= np.load("/content/drive/MyDrive/GOD-NEW-3p/latents/t_te_best.npy")
w_tr= np.load("/content/drive/MyDrive/GOD-NEW-3p/latents/t_tr_best.npy")



In [ ]:
#Load indices for filtering
file_path = "/content/drive/MyDrive/GOD-NEW-3p/Neural-DecodingPartProgresive/indeces_filtered_clipped3.dat"
importantind = np.load(file_path,allow_pickle=True)


In [ ]:
def z_score(x_tr,x_te_pt):
  # Z-score normalization
  # Calculate mean and standard deviation on training data
  norm_mean_x = np.mean(x_tr, axis=0)
  norm_std_x = np.std(x_tr, axis=0, ddof=1)
  # Avoid division by zero
  norm_std_x[norm_std_x == 0] = 1

  # Normalization
  x_tr_normalized = (x_tr - norm_mean_x) / norm_std_x
  x_te_pt_normalized = (x_te_pt - norm_mean_x) / norm_std_x

  # Z-scoring the whole dataset
  #x_whole_normalized = (x - np.mean(x, axis=0)) / np.std(x, axis=0, ddof=1)

  return x_tr_normalized,x_te_pt_normalized

## The code bellow can be optimised and without repetitions with a function. I decided that for me it is easier to look at each region separately.

#LOC

In [ ]:
#predic LOC
model_loc = LinearRegression().fit(x_tr[4], w_tr)
y_te_loc = model_loc.predict(x_te[4]) # putting as input -> LOC activity
y_tr_loc = model_loc.predict(x_tr[4])
y_te_loc.shape, y_tr_loc.shape

#y_tr_loc,y_te_loc=z_score(y_tr_loc,y_te_loc)

file_path = "/content/drive/MyDrive/GOD-NEW-3p/Plain-Z-scored/"

#Store latents
with open(f'{file_path}progressive_y_te_loc.npy', 'wb') as fp:
            pickle.dump(y_te_loc, fp)

#V4

In [ ]:
#The concatenation of the predicted latents with the next region
x_tr_v4 = np.concatenate((x_tr[3], y_tr_loc), axis=1)
x_te_v4 = np.concatenate((x_te[3], y_te_loc), axis=1)
x_tr_v4.shape, x_te_v4.shape


((1200, 1795), (50, 1795))

In [ ]:
#train a model and predict V4
model_v4 = LinearRegression().fit(x_tr_v4, w_tr)
y_te_v4 = model_v4.predict(x_te_v4) # putting as input -> V4 activity
y_tr_v4 = model_v4.predict(x_tr_v4)
y_te_v4.shape, y_tr_v4.shape

#y_tr_v4,y_te_v4=z_score(y_tr_v4,y_te_v4)

#Store latents
with open(f'{file_path}progressive_y_te_v4.npy', 'wb') as fp:
            pickle.dump(y_te_v4, fp)

#V3

In [ ]:
#The concatenation of the predicted latents with the next region
x_tr_v3 = np.concatenate((x_tr[2], y_tr_v4), axis=1)
x_te_v3 = np.concatenate((x_te[2], y_te_v4), axis=1)
x_tr_v3.shape, x_te_v3.shape



((1200, 2570), (50, 2570))

In [ ]:
#train a model and predict V3
model_v3 = LinearRegression().fit(x_tr_v3, w_tr)
y_te_v3 = model_v3.predict(x_te_v3) # putting as input -> V4 activity
y_tr_v3 = model_v3.predict(x_tr_v3)
y_te_v3.shape, y_tr_v3.shape

#y_tr_v3,y_te_v3=z_score(y_tr_v3,y_te_v3)

#Store latents
with open(f'{file_path}progressive_y_te_v3.npy', 'wb') as fp:
            pickle.dump(y_te_v3, fp)

#V2


In [ ]:
#The concatenation of the predicted latents with the next region
x_tr_v2 = np.concatenate((x_tr[1], y_tr_v3), axis=1)
x_te_v2 = np.concatenate((x_te[1], y_te_v3), axis=1)
x_tr_v2.shape, x_te_v2.shape

((1200, 3199), (50, 3199))

In [ ]:
#train a model and predict V2
model_v2 = LinearRegression().fit(x_tr_v2, w_tr)
y_te_v2 = model_v2.predict(x_te_v2)
y_tr_v2 = model_v2.predict(x_tr_v2)
y_te_v2.shape, y_tr_v2.shape


#y_tr_v2,y_te_v2=z_score(y_tr_v2,y_te_v2)

#Store latents
with open(f'{file_path}progressive_y_te_v2.npy', 'wb') as fp:
            pickle.dump(y_te_v2, fp)

#V1

In [ ]:
#The concatenation of the predicted latents with the next region
x_tr_v1 = np.concatenate((x_tr[0], y_tr_v2), axis=1)
x_te_v1 = np.concatenate((x_te[0], y_te_v2), axis=1)
x_tr_v1.shape, x_te_v1.shape

((1200, 3033), (50, 3033))

In [ ]:
#train a model and predict V1
model_v1 = LinearRegression().fit(x_tr_v1, w_tr)
y_te_v1 = model_v1.predict(x_te_v1)
y_tr_v1 = model_v1.predict(x_tr_v1)
y_te_v1.shape, y_tr_v1.shape

#y_tr_v1,y_te_v1=z_score(y_tr_v1,y_te_v1)

#train a model and predict V1
with open(f'{file_path}progressive_y_te_v1.npy', 'wb') as fp:
            pickle.dump(y_te_v1, fp)